Analyses from SWDB are in V1 file


Copied everything over so that I can change function definitions without breaking everything (2017-09-18)

Copied again for (hopefully!) finalized analysis (2017-10-12)

New version with newest ephys data (2017-10-25)

Re-analyze using newest ephys & RNA-seq data downloaded 2018-06-19 (many more cells in RNA-seq set)

Re-do using linear model instead of partial correlations (started 2018-09-25)

Run analysis on PCA-transformed data with same layer splits, also removed zero filtering step (started 2019-03-21)

Run on inhibitory neurons only as an alternative means of dealing with the class-driven correlation problem (2019-04-11)

In [1]:
# Imports

import sys
import os

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.sandbox.stats.multicomp import fdrcorrection0
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from matplotlib_venn import venn2
from matplotlib_venn import venn2_circles
import seaborn as sns
%matplotlib inline


//anaconda/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# fit linear models explaining ephys according to gene expression and/or cell class
# returns a summary dataframe containing beta values, class-specific slopes, and p-values

def fit_models(df_seq, df_var2, line_labels, min_samples = 10):
    
    measures = df_var2.index
    genes = df_seq.index
        
    result_list = []
    
    line_labels = pd.DataFrame(line_labels, index = df_var2.columns, columns = ['cell_class'])

    df_seq = df_seq.loc[:, line_labels.cell_class == 'inh']
    df_var2 = df_var2.loc[:, line_labels.cell_class == 'inh']
    
    # For each combination of gene and measure, pull out data into x and y variables
    for n, measure in enumerate(measures):
        for m, gene in enumerate(genes):
            if (gene in df_seq.index) & ((df_seq.loc[gene] > 1).sum() >= min_samples):
                x = df_seq.loc[gene, :]
                y = df_var2.loc[measure, :]
                
                # If shapes of x and y make sense, create models
                if x.shape == (df_var2.shape[1],) and y.shape == (df_var2.shape[1],):

                    # Make a data frame with x and y data
                    df_int = pd.concat([x, y], axis = 1)
                    gene_idx = 'Gene' + str(gene)
                    df_int.rename(columns = {gene: gene_idx}, inplace = True)

                    # Calculate models
                    mod1 = smf.ols(formula = measure + ' ~ ' + ' +  ' + gene_idx, data = df_int)
                    res1 = mod1.fit()

                    results = [measure, gene, 
                               res1.params[gene_idx],
                               res1.pvalues[gene_idx], 
                               res1.aic]

                    result_list.append(results)
                    
    # Convert to dataframe
    df = pd.DataFrame(result_list, columns = ['property', 'gene_entrez_id', 
                                              'beta_gene_inh_only',  
                                              'pval_gene_inh_only', 
                                              'inh_only_aic'])
    return df

In [3]:
# Load ephys data

input_folder_1 = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2018-10-09/'
input_folder_2 = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2019-03-21 PCA/'

df_seq_ephys = pd.read_csv(input_folder_1 + 'seq_mean.csv', index_col = 0)
df_ephys = pd.read_csv(input_folder_1 + 'ephys_mean.csv', index_col = 0)
df_ephys_pca = pd.read_csv(input_folder_2 + 'ephys_mean_pca.csv', index_col = 0)
df_ephys = pd.concat([df_ephys, df_ephys_pca])

# Remove raw data for features that were log transformed
ephys_features = ['rmp', 'tau', 'apthr', 'apamp', 'ahpamp', 'aphw', 'rheo', 'cap', 'maxfreq', 
                  'adratio', 'ri', 'sag', 'f_i_curve_slope', 'avg_isi', 'latency', 'isi_cv']
for measure in ['ri', 'tau', 'cap', 'rheo', 'maxfreq', 'ahpamp', 'adratio', 
                'f_i_curve_slope', 'avg_isi', 'latency', 'isi_cv', 'sag']:
    ephys_features.remove(measure)
    ephys_features.append(measure + '_log10')
ephys_features = ephys_features + ['PC1', 'PC2', 'PC3']
df_ephys = df_ephys.loc[ephys_features]

In [4]:
# Load morphology data

input_folder_1 = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2018-10-16/'
input_folder_2 = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2019-03-21 PCA/'

df_seq_morph = pd.read_csv(input_folder_1 + 'seq_mean.csv', index_col = 0)
df_morph = pd.read_csv(input_folder_1 + 'morph_mean.csv', index_col = 0)
df_morph_pca = pd.read_csv(input_folder_2 + 'morph_mean_pca.csv', index_col = 0)
df_morph = pd.concat([df_morph, df_morph_pca])

morph_features = ['branchiness', 'average_bifurcation_angle_local', 'max_branch_order', 
                  'soma_surface', 'total_length', 'total_volume']
for measure in ['branchiness', 'max_branch_order', 'total_length', 'total_volume']:
    morph_features.remove(measure)
    morph_features.append(measure + '_log10')
morph_features = morph_features + ['PC1', 'PC2', 'PC3']
df_morph = df_morph.loc[morph_features]

In [5]:
path_seq = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/mouse_VISp_gene_expression_matrices_2018-06-14/'
filename_genes = 'mouse_VISp_2018-06-14_genes-rows.csv'
gene_info = pd.read_csv(path_seq + filename_genes)

In [6]:
output_folder_ephys = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2019-04-11 Interneurons only Ephys/'
output_folder_morph = '/Users/Claire/Dropbox/Allen_Institute_Stuff/Ephys_GE/Analysis/2019-04-11 Interneurons only Morphology/'

<h3>OLS regression analysis</h3>

In [7]:
for df_seq, df_var2, output_folder, label in [(df_seq_ephys, df_ephys, output_folder_ephys, 'ephys'), 
                                              (df_seq_morph, df_morph, output_folder_morph, 'morph')]:

    if not all(df_var2.columns == df_seq.columns):
        print('Warning: Check inputs!')

    line_labels = [line.split('__')[-1] for line in df_var2.columns]

    features = df_var2.index

    df = fit_models(df_seq, df_var2, line_labels)

    ptype = 'pval_gene_inh_only'
    q_list_all = []
    mask = np.isfinite(df[ptype])
    for measure in features:
        p_list = np.array(df[mask][df[mask].property == measure][ptype])
        _, q_list = fdrcorrection0(p_list, alpha = 0.05)
        q_list_all.append(q_list)
    df[ptype.replace('pval', 'FDR')] = np.nan
    df.loc[mask, ptype.replace('pval', 'FDR')] = np.hstack(q_list_all)

    # add gene symbols
    df.insert(2, 'gene_symbol', [gene_info[gene_info.gene_entrez_id == n]['gene_symbol'].values[0] for n in df.gene_entrez_id])

    # Save
    df.to_csv(output_folder + 'results_table.csv')

    # percentage of genes with acceptable expression
    print(((float(df.shape[0]) / len(features)) / float(df_seq.shape[0])) * 100)
    

25.2753015207
24.9956301346
